In [1]:
import os
import yaml
from pathlib import Path
import cv2
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Flatten
from tensorflow.keras.preprocessing import image_dataset_from_directory

2025-02-20 21:52:03.602166: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-20 21:52:04.135564: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-20 21:52:04.288558: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-02-20 21:52:04.289293: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-20 21:52:04.492334: I tensorflow/core/platform/cpu_feature_gua

In [54]:


# EDA: Displaying the class distribution
def load_labels(path):
    labels = []
    for label_file in Path(path).glob('*.txt'):
        with open(label_file, 'r') as file:
            for line in file:
                labels.append(int(line.split()[0]))
    return labels

# Load labels for train, validation, and test sets
train_labels = load_labels(train_path.replace('images', 'labels'))
val_labels = load_labels(val_path.replace('images', 'labels'))
test_labels = load_labels(test_path.replace('images', 'labels'))

# Combine labels and create a count series
labels = train_labels + val_labels + test_labels
labels_count = pd.Series(labels).value_counts().sort_index()
labels_count.index = [class_names[i] for i in labels_count.index]

# Plot class distribution
fig = px.bar(labels_count, title="Class Distribution", labels={'index': 'Class', 'value': 'Count'},
             template='plotly_dark', color=labels_count, color_continuous_scale='reds')
fig.show()



In [56]:
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(96, 96, 3))
base_model.trainable = False

In [57]:
model = Sequential()
model.add(base_model)
#model.add(Conv2D(kernel_size=(3,3)))
model.add(Flatten())
model.add(Dense(6,activation='relu'))
model.add(Dense(1,activation='softmax'))

In [58]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['Accuracy'])

history = model.fit(data_yaml_path,epochs=20)

ValueError: Unrecognized data type: x=data.yaml (of type <class 'str'>)